In [4]:
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os
import numpy as np
import pandas as pd
import gc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from keras import layers
from keras import models
from keras import optimizers
from keras import losses
from keras import metrics
from keras.models import load_model

from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
# import flax
# function to make dataframe from .arff file
def load(f):
 with open(f, 'r') as fp:
    try:
      file_content = fp.readlines()
    except:
      return "NOT READABLE"


 def parse_row(line, len_row):
    line = line.replace('{', '').replace('}', '')

    row = np.zeros(len_row)
    d=line.split(',')
    if(len(d)!=1):
     for data in line.split(','):
         index, value = data.split()
         row[int(index)] = float(value)

    return row


 columns = []
 len_attr = len('@attribute')
 

# get the columns
 for line in file_content:
    if line.startswith('@attribute '):
        col_name = line[len_attr:].split()[0]
        columns.append(col_name)
 rows = []
 len_row = len(columns)
# get the rows
 for line in file_content:
    if line.startswith('{'):
        
        rows.append(parse_row(line, len_row))

 df = pd.DataFrame(data=rows, columns=columns)
 # give path of training (   ......Recon) or test file(/content/drive/MyDrive/arff_manual2/ ...our data)
 domain_os=f.replace("D:\\Explainable_AI-main\\arff_all","")
 domain_os=domain_os.replace(".arff","")
 temp=domain_os.split("_")
 domain=temp[0]
 os=temp[1]
 fo=[]
 fo1=[]
 for i in range(0,len(df.index)):
   fo.append(domain)
   fo1.append(os)

 df.insert(1,"domain",fo)
 df.insert(2,"OS",fo1)

 return df
# making final data frame by combining all training dataframes (per domain)

# Combing columns
col = []
coli = []
rows = []
count = 500
for file in os.listdir("D:\\Explainable_AI-main\\arff_all"):
  #print(file)
  if count == 0:
    break
  count = count - 1
  if file == "general_all.arff":
    continue

  s = "D:\\Explainable_AI-main\\arff_all\\" + file
  df1 = load(s)
  if type(df1) != pd.core.frame.DataFrame:
    continue

  col1 = df1.columns
  col = np.concatenate((col, col1))
  if len(coli) == 0:
    coli = col1
  else:
    coli = np.intersect1d(coli, col1)

col = np.unique(col)
# print(col)
# print(coli)
# print(len(col))


# Combining rows
count = 500
for file in os.listdir("D:\\Explainable_AI-main\\arff_all"):
  if count == 0:
    break
  count = count - 1
  if file == "general_all.arff":
    continue
  # print(file)
  s = "D:\\Explainable_AI-main\\arff_all\\" + file
  df1 = load(s)
  if type(df1) != pd.core.frame.DataFrame:
    continue
  col1 = df1.columns
  for index, row in df1.iterrows():
    map = {}
    for c in col1:
      map[c] = row[c]

    ro = []
    for i in range(0, len(col)):
      ro.append("0.0")

    for i in range(0, len(col)):
      t = col[i]
      if t in map:
        ro[i] = map[t]

    rows.append(ro)
 
      
   
# Chnaging position for y column to last       
df = pd.DataFrame(data=rows, columns=col)
#df=df.fillna(0.0)
col2=col.copy
c=df.pop("PIILabel")
d=df.pop("domain")
e=df.pop("OS")
df.insert(len(df.columns), 'PIILabel', c)
df.insert(0, 'domain', d)
df.insert(1, 'OS', e)

df
np.unique(df['PIILabel'], return_counts = True)
!mkdir combined_data_stuff
imp_features = pd.read_csv("/media/rishika/Seagate Expansion Drive/shreyas_btp_pii_leakage/combined_data_stuff/imp_features_rp_heuristic.csv").drop("Unnamed: 0", axis = 1)
df_wod = df.drop(["domain","OS"], axis = 1)
c_data = []
for i in df_wod.columns:
    no_pkts_1 = 0
    no_pkts_pil = 0
    for j in df_wod.index:
        if(df_wod[i][j] != 0):
            if(df_wod["PIILabel"][j] != 0):
                no_pkts_pil = no_pkts_pil + 1
            no_pkts_1 = no_pkts_1 + 1
    confidence = no_pkts_pil/no_pkts_1
    c_data.append([i, confidence])
c_df = pd.DataFrame(c_data)
c_df.rename(columns = {0:'col_name', 1:'confidence_level'}, inplace = True)
c_df_imp = c_df[c_df.confidence_level > 0.17]
c_df_imp
c_df_imp.to_csv("combined_data_stuff/imp_features_rp_heuristic.csv")
imp_cols = list(c_df_imp['col_name'])
undesired_columns=[]
for i in df_wod.columns:
    if i not in imp_cols:
        undesired_columns.append(i)
heuristic_df= df_wod.drop(undesired_columns, axis=1)
heuristic_df.shape
heuristic_df.head()
def convert_bin(x):
    if x > 0:
        return 1
    else:
        return 0
bin_values = heuristic_df.values.astype('float32')
conv_bin = np.vectorize(convert_bin)
bin_values = conv_bin(bin_values)
bin_df = pd.DataFrame(bin_values, columns = heuristic_df.columns)
bin_df.to_csv("combined_data_stuff/bin_data_full.csv")
bin_df.head()
bin_df = pd.read_csv('combined_data_stuff/bin_data_full.csv').drop(['Unnamed: 0'],axis=1)
bin_df.head()
X = bin_df.iloc[:, :-1]
y = bin_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_train_data, x_val_data, y_train_data, y_val_data = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
model=models.Sequential()
model.add(layers.Dense(x_train_data.shape[1],activation='relu',input_shape=(x_train_data.shape[1],)))
model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()
import time
st = time.time()
model.fit(x_train_data,y_train_data,epochs=5,batch_size=32,validation_data=(x_val_data,y_val_data))
end = time.time()
time_taken_nn_train = end-st
print("Time taken to train NN = ", time_taken_nn_train, "seconds")
print("score of NN on train: "+ str(model.evaluate(X_train,y_train)[1]))
print("score of NN on test: " + str(model.evaluate(X_test,y_test)[1]))
import matplotlib.pyplot as plt

weights= model.layers[0].get_weights()[0]
weights = weights.reshape(-1)
#print(weights)
plt.hist(weights)
plt.colorbar()
plt.show()
# model.save("combined_data_stuff/nn_weights.h5")
clf = DecisionTreeClassifier()
st = time.time()
clf = clf.fit(x_train_data,y_train_data)
end = time.time()
time_taken_dt_train = end-st
print("Time taken to train DT = ", time_taken_dt_train, "seconds")
y_pred_test = clf.predict(X_test)
y_pred_train = clf.predict(X_train)
print("score of DT on train: "+ str(accuracy_score(y_pred_train,y_train)))
print("score of DT on test: " + str(accuracy_score(y_pred_test,y_test)))
model_2=models.Sequential()
model_2.add(layers.Dense(x_train_data.shape[1],activation='relu',input_shape=(x_train_data.shape[1],)))
model_2.add(layers.Dense(1024,activation='relu'))
model_2.add(layers.Dense(256,activation='relu'))
model_2.add(layers.Dense(64,activation='relu'))
model_2.add(layers.Dense(1,activation='sigmoid'))
model_2.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model_2.summary()
model_2 = load_model('combined_data_stuff/nn_weights.h5')
print("score of NN on train: "+ str(model_2.evaluate(X_train,y_train)[1]))
print("score of NN on test: " + str(model_2.evaluate(X_test,y_test)[1]))
import lime
from lime import lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(x_train_data.values,
                                                   feature_names=x_train_data.columns,
                                                   class_names=['Non-PII', 'PII'],
                                                   discretize_continuous=True)

instance_idx = 0

def predict_proba_wrapper(X):
    # Make sure to modify 'model' to your TensorFlow Sequential model
    preds = model.predict(X)
    # If only one class is present in the data, create a placeholder probability for the missing class
    if preds.shape[1] == 1:
        preds = np.concatenate([preds, 1 - preds], axis=1)
    return preds

explanation = explainer.explain_instance(x_val_data.values[instance_idx],
                                         predict_proba_wrapper, num_features=len(x_train_data.columns))


#fig = explanation.as_pyplot_figure()
pd.DataFrame(explanation.as_list())


# Display the figure


#explanation.show_in_notebook(show_table=True, show_all=False)


NotADirectoryError: [WinError 267] The directory name is invalid: 'D:\\Explainable_AI-main\\arff_all'

In [ ]:
import lime
from lime import lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(x_train_data.values,
                                                   feature_names=x_train_data.columns,
                                                   class_names=['Non-PII', 'PII'],
                                                   discretize_continuous=True)

instance_idx = 0

def predict_proba_wrapper(X):
    # Make sure to modify 'model' to your TensorFlow Sequential model
    preds = model.predict(X)
    # If only one class is present in the data, create a placeholder probability for the missing class
    if preds.shape[1] == 1:
        preds = np.concatenate([preds, 1 - preds], axis=1)
    return preds

explanation = explainer.explain_instance(x_val_data.values[instance_idx],
                                         predict_proba_wrapper, num_features=len(x_train_data.columns))


#fig = explanation.as_pyplot_figure()
pd.DataFrame(explanation.as_list())


# Display the figure


#explanation.show_in_notebook(show_table=True, show_all=False)
import shap
import numpy as np

# Convert validation data to numpy array
x_test_np = X_test.to_numpy()

# Create the SHAP explainer using TFDeepExplainer and a random sample of the validation data as the background dataset
background_data_sample = x_test_np[np.random.choice(x_test_np.shape[0], 100, replace=False)]
explainer = shap.DeepExplainer(model, background_data_sample)

# Compute SHAP values for the validation data
shap_values = explainer.shap_values(x_test_np)

# Plot the SHAP summary plot
shap.summary_plot(shap_values, x_test_np, plot_type="bar")


In [ ]:
import shap
import numpy as np

# Convert validation data to numpy array
x_test_np = X_test.to_numpy()

# Create the SHAP explainer using TFDeepExplainer and a random sample of the validation data as the background dataset
background_data_sample = x_test_np[np.random.choice(x_test_np.shape[0], 100, replace=False)]
explainer = shap.DeepExplainer(model, background_data_sample)

# Compute SHAP values for the validation data
shap_values = explainer.shap_values(x_test_np)

# Plot the SHAP summary plot
shap.summary_plot(shap_values, x_test_np, plot_type="bar")
